In [ ]:
# Cost of electrochemical cell
# ignore electronics, piping, valves - just want basic material cap cost?
# each cell has 2 endplates, 2 current collectors, gde, zinc
# find cost per cell - do zinc separately
cell_area = 1 #m2
cell_cost = cell_area * (2*cost_endplate + 2*cost_curr_coll + cost_GDE)

# cost of manifold? - depeds on number of cells
def manifold(num_cells)
    pass

cost_manifold = manifold(num_cells)

# cost of stack
stack_cost = cost_manifold + num_cells*cell_cost

# cost of air blower
# C = a + bS^n
blower_a = 
blower_cost = blower_a + blower_b*blower_s**blower_n